In [ ]:
import numpy as np
import matplotlib.pyplot as plt

% matplotlib inline

# Otimização

Nesta aula estudaremos alguns métodos de minimização de funções de uma variável. Dada uma função $f(x)$, nosso objetivo é encontrar $x$ que produza o menor valor possível para $y = f(x)$. Este tipo de problema aparece em diversas áreas da matemática e engenharia, ainda que normalmente na versão multivariada que veremos em outra ocasião.

Considere como exemplo a função $f(x) = -x e^{-x}$. Vemos no gráfico que ela claramente possui um mínimo próximo de $x = 1$.

In [ ]:
def f(x):
    return -x * np.exp(-x)

X = np.linspace(0, 10, 100)
Y = f(X)
plt.plot(X, Y)

Podemos mostrar analiticamente que o mínimo está de fato no ponto $x = 1$, basta derivar e igualar a zero:

$$
\begin{align}f'(x) &= -e^{-x} + x e^{-x} = 0 \\ 
                   & \Rightarrow x = 1
\end{align}
$$

Dependendo da função, o cálculo da derivada pode ser bastante complicado ou a etapa de isolar $x$ quando a derivada é igualada à zero pode ser inviável de se fazer analiticamente. Para isto, vamos mostrar alguns métodos numéricos que podem ser aplicados sem utilizar estes passos complicados.

## Busca ternária

Vamos supor que não saibamos que o mínimo esteja exatamente no ponto $x = 1$, mas temos uma idéia razoável que ele esteja dentro do intervalo $x\in[0,10]$ (como se pode ver claramente pelo gráfico).

Iniciamos nossa busca fixando estes dois pontos avaliando dois pontos dentro deste intervalo. A estratégia lembra um pouco o método da bisseção, com a diferença que guardamos sempre os valores dos extremos dos intervalos junto com um valor no centro menor que ambos os extremos (se o valor do centro for maior que algum dos extremos, significa que a função possui mais de um mínimo e não pode ser tratada por este método).

A figura mostra o primeiro passo do algoritmo

In [ ]:
# Prepara pontos no interior do intervalo
x1 = 0
x2 = 10
xa = x1 + (x2 - x1)/3
xb = x2 - (x2 - x1)/3

# Gráfico
plt.plot(X, Y)
plt.xticks([x1, xa, xb, x2], ['$x_1$','$x_a$', '$x_b$', '$x_2$'])
xmin, xmax, ymin, ymax = plt.axis()
for x in [x1, xa, xb, x2]:
    plt.plot([x, x], [ymin, ymax], 'k--')
    plt.plot([x], [f(x)], 'ko')

Vemos que dos dois novos pontos, $x_a$ possui um valor menor. Assim, atualizamos nosso intervalo para que $x_b$
seja promovido ao extremo direito e $x_a$ continue dentro do intervalo. Desta forma, no passo seguinte do algoritmo temos:

In [ ]:
# Prepara pontos no interior do intervalo
x1 = 0
x2 = xb                # x2 vira o antigo xb!
xa = x1 + (x2 - x1)/3  # atualizamos os novos pontos dentro do intervalo
xb = x2 - (x2 - x1)/3

# Gráfico
plt.plot(X, Y)
plt.xticks([x1, xa, xb, x2], ['$x_1$','$x_a$', '$x_b$', '$x_2$'])
xmin, xmax, ymin, ymax = plt.axis()
for i, x in enumerate([x1, xa, xb, x2]):
    plt.plot([x, x], [ymin, ymax], ('k--' if i in [1, 2] else 'k-'))
    plt.plot([x], [f(x)], 'ko')

O algoritmo segue assim por mais alguns passos. Devemos sempre reduzir o intervalo escolhendo o *maior* ponto entre os dois pontos escolhidos em cada etapa.

In [ ]:
for i in range(7):
    xa = x1 + (x2 - x1)/3
    xb = x2 - (x2 - x1)/3

    # Primeiro fazemos o gráfico
    plt.plot(X, Y)
    plt.xticks([x1, xa, xb, x2], ['$x_1$','$x_a$', '$x_b$', '$x_2$'])
    xmin, xmax, ymin, ymax = plt.axis()
    for i, x in enumerate([x1, xa, xb, x2]):
        plt.plot([x, x], [ymin, ymax], ('k--' if i in [1, 2] else 'k-'))
        plt.plot([x], [f(x)], 'ko')
    plt.show()
        
    # Agora atualizamos o resultado
    fa = f(xa)
    fb = f(xb)
    if fa > fb:
        x1 = xa
    else:
        x2 = xb

Veja como o intervalo vai gradualmente encolhendo em torno do mínimo.

## Implementação

Vamos agora implementar a busca ternária mais formalmente como uma função chamada `ternaria()` que recebe uma função objetivo e um intervalo e retorna o mínimo da função. A estratégia é continuar a iteração até que o intervalo seja suficientemente pequeno.

In [ ]:
def ternaria(f, x1, x2, xtol=1e-6):
    """
    Realiza a busca ternária pelo mínimo da função f(x) data dentro do intervalo [x1, x2].
    
    Args:
        f: função objetivo f(x).
        x1, x2: início e fim do intervalo de busca.
        xtol: tolerância no intervalo de busca.
        
    Returns:
        Retorna o mínimo quando o tamanho do intervalo de busca for menor que xtol.
    """
    
    while True:
        xa, xb = x1 + (x2 - x1)/3, x1 + (x2 - x1) * 2 / 3
        fa, fb = f(xa), f(xb)
        
        if fa > fb:
            x1 = xa
        else:
            x2 = xb
        
        if abs(x1 - x2) < xtol:
            break
    
    return xa if fa < fb else xb

Agora testamos com a nossa função exemplo e algumas outras funções conhecidas

In [ ]:
ternaria(f, 0, 10)

In [ ]:
ternaria(np.cos, 0, 2 * np.pi)

## Busca pela razão áurea

Na busca ternária, dividimos o intervalo em três partes iguais. Ainda que isto pareça intuitivo, não é a melhor opção. Um problema da busca ternária é que a cada iteração precisamos avaliar a função em dois pontos. Se fizermos a escolha adequada, é possível avaliar a função em apenas um ponto e reciclar o ponto anterior. Para que isto ocorra, é necessário dividir o intervalo em partes proporcionais à razão áurea $phi$. 

In [ ]:
phi = (1 + np.sqrt(5)) / 2
x1, x2 = 0, 10

for i in range(7):
    xa = x2 - (x2 - x1)/phi
    xb = x1 + (x2 - x1)/phi
    
    # Primeiro fazemos o gráfico
    plt.plot(X, Y)
    plt.xticks([x1, xa, xb, x2], ['$x_1$','$x_a$', '$x_b$', '$x_2$'])
    xmin, xmax, ymin, ymax = plt.axis()
    for i, x in enumerate([x1, xa, xb, x2]):
        plt.plot([x, x], [ymin, ymax], ('k--' if i in [1, 2] else 'k-'))
        plt.plot([x], [f(x)], 'ko')
    plt.show()
        
    # Agora atualizamos o resultado
    fa = f(xa)
    fb = f(xb)
    if fa > fb:
        x1 = xa
    else:
        x2 = xb

Note que uma das linhas em $x_a$ ou $x_b$ sempre se alinha com um dos $x_a$ e $x_b$ da iteração posterior. Isto é proposital e permite que avaliemos a função apenas em um dos pontos interiores ao novo intervalo de iteração. 

In [ ]:
phi = (1 + np.sqrt(5)) / 2

def razao_aurea(f, x1, x2, xtol=1e-6):
    """
    Retorna o mínimo da função f(x) data dentro do intervalo [x1, x2] utilizando a busca pela razão áurea.
    
    Args:
        f: função objetivo f(x).
        x1, x2: início e fim do intervalo de busca.
        xtol: tolerância no intervalo de busca.
        
    Returns:
        Retorna o valor de x que minimiza f(x) quando o tamanho do intervalo de busca for menor que xtol.
    """
    
    xa = x2 - (x2 - x1)/phi
    xb = x1 + (x2 - x1)/phi
    fa, fb = f(xa), f(xb)
    update = None
    
    while True:
        if fa > fb:
            x1 = xa
            xa, fa = xb, fb
            xb = x1 + (x2 - x1)/phi
            fb = f(xb)
        else:
            x2 = xb
            xb, fb = xa, fa
            xa = x2 - (x2 - x1)/phi
            fa = f(xa)
        
        if abs(x1 - x2) < xtol:
            break
            
    return xa if fa < fb else xb

Agora testamos alguns resultados

In [ ]:
razao_aurea(f, 0, 10)

In [ ]:
razao_aurea(np.cos, 0, 2*np.pi)

Finalmente, comparamos a performance de cada método.

In [ ]:
%timeit -n1000 ternaria(f, 0, 10)
%timeit -n1000 razao_aurea(f, 0, 10)

In [ ]:
%timeit -n1000 ternaria(np.cos, 0, 2*np.pi)
%timeit -n1000 razao_aurea(np.cos, 0, 2*np.pi)

Vemos que nos dois casos a razão áurea oferece quase o dobro da performance da busca ternária. Isto ocorre porque reaproveitamos uma avaliação da função por iteração.

## Descendo o gradiente

Uma outra estratégia de minimização consiste em olhar a derivada da função e decidir, a partir desta informação,
para qual direção devemos procurar o mínimo. Como a derivada fornece a direção de crescimento, devemos andar sempre para a esquerda se a derivada for positiva (função crescendo no ponto) ou para a direita se a derivada for negativa.

Vamos decidir sobre um passo e realizar algumas iterações desta estratégia:

In [ ]:
def f_linha(x):
    return (x  - 1) * np.exp(-x)

# Calculamos vários valores de x obtidos com a estratégia de andar na direção oposta à derivada
x = 5
passo = 0.3
X = [x]
for i in range(30):
    deriv = f_linha(x)
    if deriv > 0:
        x -= passo
    else:
        x += passo
    X.append(x)
    
# Fazemos um gráfico dos resultados
plt.plot(X, 'ko--', label='aproximação via derivada')
plt.plot([1] * len(X), 'r-', lw=2, label='valor correto')
plt.legend(loc='best')

Temos um problema! O algoritmo anda na direção correta até chegar próximo ao mínimo. A partir daí, começa a oscilar entre dois pontos em torno do mínimo sem nunca se aproximar verdadeiramente do mínimo. O motivo para que isto aconteça é que o passo é fixo. Talvez fosse melhor que o passo fosse proporcional ao próprio valor da derivada. Desta forma, quando a derivada fosse menor que zero, daríamos passos menores para aproximar mais lentamente do mínimo.

In [ ]:
x = 5
alpha = 3.0
X = [x]
for i in range(30):
    deriv = f_linha(x)
    passo = -alpha * deriv
    x += passo
    X.append(x)
    
# Fazemos um gráfico dos resultados
plt.plot(X, 'ko--', label='aproximação via derivada')
plt.plot([1] * len(X), 'r-', lw=2, label='valor correto')
plt.legend(loc='best')

Bem melhor!

Agora queremos diminuir o número de iterações calibrando melhor o tamanho de cada passo. Vemos que é possível dar passos bem maiores que utilizamos para construir a figura acima. Isto pode ser feito controlando a variável alpha. No entanto, o ideal é que o próprio alpha evolua para que seja possível utilizar valores altos quando a função é mais previsível e valores menores quando uma maior precisão for necessária. 

Nossa estratégia é aumentar o valor de alpha a cada iteração e reduzí-lo sempre que o passo tente mandar para um novo ponto que aumenta o valor da função ao invés de diminuí-lo.

In [ ]:
x = 5
alpha = 3.0
r_mais = 1.5
r_menos = 0.5
fmin = f(x)

X = [x]
for i in range(30):
    deriv = f_linha(x)
    passo = -alpha * deriv
    
    # Limita a 10 iterações
    for i in range(10):
        fnew = f(x + passo)
        if fnew < fmin:
            fmin = fnew
            break
        else:
            alpha *= r_menos
            passo = -alpha * deriv
        
    x += passo
    alpha *= r_mais
    
    X.append(x)
    Y.append(f(x))
    
# Fazemos um gráfico dos resultados
plt.plot(X, 'ko--', label='aproximação via derivada')
plt.plot([1] * len(X), 'r-', lw=2, label='valor correto')
plt.legend(loc='best')
plt.show()
print('xmin:', x)

Enquanto a primeira versão pedia 20 iterações esta se aproxima bastante do resultado com apenas 10 iterações.
Vamos à implementação.

In [ ]:
def gradiente(f, f_linha, x0, xtol=1e-6, dtol=1e-6, alpha=5, r_mais=1.5, r_menos=0.25):
    """
    Retorna o mínimo da função f(x) com derivada f_linha fazendo uma descida de gradiente.
    
    Args:
        f: função objetivo f(x).
        f_linha: derivada de f(x).
        x1, x2: início e fim do intervalo de busca.
        xtol: tolerância no intervalo de busca.
        dtol: tolerância na derivada.
        alpha: constante que define tamanho inicial do passo.
        r_mais: fator de expansão do passo a cada iteração (r_mais >= 1).
        r_menos: fator de redução do passo a cada iteração mal sucedida (r_menos <= 1).
        
    Returns:
        Retorna o valor de x que minimiza f(x).
    """
    x = x0
    fmin = f(x)
    while True:
        deriv = f_linha(x)
        if abs(deriv) < dtol:
            return x
        passo = -alpha * deriv

        # Limita a 10 iterações de redução do alpha
        for i in range(10):
            fnew = f(x + passo)
            if fnew < fmin:
                fmin = fnew
                break
            else:
                alpha *= r_menos
                passo = -alpha * deriv

        x += passo
        alpha *= r_mais
        if abs(passo) < xtol:
            return x

Testamos para funções conhecidas

In [ ]:
gradiente(f, f_linha, 5, dtol=0), gradiente(np.cos, lambda x: -np.sin(x), 5, dtol=0)

E comparamos a performance...

In [ ]:
%timeit -n1000 razao_aurea(f, 0, 10)
%timeit -n1000 gradiente(f, f_linha, 5, dtol=0)

In [ ]:
%timeit -n1000 razao_aurea(f, 0, 10, xtol=1e-10)
%timeit -n1000 gradiente(f, f_linha, 5, xtol=1e-10, dtol=0)

Vemos que o método da descida do gradiente não supera a razão áurea em todos exemplos.

O principal problema talvez seja que os métodos de descida do gradiente tendem a se aproximar rapidamente de uma vizinhança do mínimo mas normalmente são relativamente lentos em finalmente atingir este valor. Isto está relacionado ao fato que cada passo é proporcional ao valor do gradiente, sendo que este tende a zero próximo do mínimo. 

Uma solução para este dilema é mudar de método: vamos estudar o método das aproximações parabólicas e posteriormente implementar uma estratégia que tente extrair o melhor de cada um.

## Método de Newton

Uma estratégia muito bem sucedida consiste em aproximar a função por parábolas e estimar o mínimo a cada iteração pelo mínimo desta parábola. Muitas vezes fazemos esta estimativa utilizando a derivada segunda da função (também chamada de hessiana).

Utilizamos para isto a expansão da função em série de Taylor:

$$f(x) \simeq f(x_0) + f'(x_0)(x - x_0) + \frac12 f''(x_0)(x - x0)^2$$

Tirando a derivada, calculamos o mínimo desta parábola

$$f'(x_0) + f''(x_0)(x - x_0) = 0$$

Ou seja:

$$x = x_0 - \frac{f'(x_0)}{f''(x_0)}$$

Os gráficos abaixo ilustram a otimização com base na Hessiana

In [ ]:
def hess(x):
    return (2 - x) * np.exp(-x)

x0 = 1.4
X = np.linspace(0, 10, 100)
for i in range(5):
    f0 = f(x0)
    f1 = f_linha(x0)
    f2 = hess(x0)
    
    plt.plot(X, f(X), 'k-', label='função')
    plt.plot(X, f0 + f1 * (X - x0) + 0.5 * f2 * (X - x0)**2, label='aproximação quadrática')
    plt.plot([x0], [f0], 'ko')
    plt.axis([0, 10, -0.5, 0.25])
    plt.axis([0, 10, -0.5, 0.25])
    plt.show()
    
    x0 -= f1 / f2
    print('xmin:', x0)
    

Observe que uma vez que o método se aproxima do mínimo, a convergência é muito rápida. Apesar disto, trata-se de um método frágil. Se não estivermos próximos do mínimo e a função não for muito bem descrita por uma parábola, o método gera valores absurdos, como pode-se experimentar escolhendo um valor de $x$ inicial longe de $x=1$.

Outro problema é a necessidade de fornecer explicitamente a Hessiana. Isto deixa o método mais trabalhoso de implementar, ainda que a performance próxima do mínimo seja muito boa.

A boa notícia é que o método de Newton e o método de descida do gradiente são complementares: o primeiro consegue se aproximar rapidamente do mínimo enquanto que o outro é muito eficiente mas falha se não for inicializado o suficientemente próximo do mínimo. Vamos tentar uma abordagem mista: fazemos a descida do gradiente normalmente, mas em alguns passos tentamos utilizar o método de Newton aproximando a Hessiana por duas avaliações seguidas da derivada.

In [ ]:
def gradiente_newton(f, f_linha, x0, xtol=1e-6, dtol=1e-6, alpha=5, r_mais=1.5, r_menos=0.25):
    """
    Retorna o mínimo da função f(x) com derivada f_linha fazendo uma descida de gradiente.
    
    Args:
        f: função objetivo f(x).
        f_linha: derivada de f(x).
        x1, x2: início e fim do intervalo de busca.
        xtol: tolerância no intervalo de busca.
        dtol: tolerância na derivada.
        alpha: constante que define tamanho inicial do passo.
        r_mais: fator de expansão do passo a cada iteração (r_mais >= 1).
        r_menos: fator de redução do passo a cada iteração mal sucedida (r_menos <= 1).
        
    Returns:
        Retorna o valor de x que minimiza f(x).
    """
    x = x0
    fmin = f(x)
    while True:
        deriv = f_linha(x)
        if abs(deriv) < dtol:
            return x
        passo = -alpha * deriv

        # Limita a 10 iterações de redução do alpha
        for i in range(10):
            fnew = f(x + passo)
            if fnew < fmin:
                fmin = fnew
                
                # Aqui chegamos a um valor possivelmente próximo do zero.
                # Vamos tentar aproximar a Hessiana pela formula abaixo
                hess = (f_linha(x + passo) - deriv) / passo
                
                # Se a hessiana for negativa, temos um máximo, caso contrário, 
                # temos um mínimo
                if hess > 0:
                    xnew = x - deriv/hess
                    f_newton = f(xnew)
                    if f_newton < fmin:
                        fmin = f_newton 
                        x = xnew - passo  # tiramos um passo pois ele será 
                                          # acrescentado posteriormente
                break
            else:
                alpha *= r_menos
                passo = -alpha * deriv

        x += passo
        alpha *= r_mais
        if abs(passo) < xtol:
            return x

Testamos o método

In [ ]:
gradiente_newton(f, f_linha, 5, dtol=0), gradiente_newton(np.cos, lambda x: -np.sin(x), 5, dtol=0)

Agora vamos analizar a performance

In [ ]:
%timeit -n1000 razao_aurea(f, 0, 10)
%timeit -n1000 gradiente(f, f_linha, 5, dtol=0)
%timeit -n1000 gradiente_newton(f, f_linha, 5, dtol=0)

In [ ]:
%timeit -n1000 razao_aurea(f, 0, 10, xtol=1e-10)
%timeit -n1000 gradiente(f, f_linha, 5, xtol=1e-10, dtol=0)
%timeit -n1000 gradiente_newton(f, f_linha, 5, xtol=1e-10, dtol=0)

Vemos então que o método torna-se mais eficaz quando exige-se uma maior precisão.